Using the images marked as valid per cluster, we pass them through the CNN and extract their feature vectors. the results are stored at a per-country basis. For example, all Malawi feature extractions will go into results/malawi_2016/cnn.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
BASE_DIR = 'gdrive/MyDrive/geo'
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'cnn_images')
CNN_DIR = os.path.join(BASE_DIR, 'models', 'trained_model.pt')

In [ ]:
os.makedirs(RESULTS_DIR, exist_ok=True)
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(RESULTS_DIR, country), exist_ok=True)

# Feature extract with CNN
If you have run this step before, you can skip it and run the commented out code in the next section to quick-start.

In [ ]:
df_images = pd.read_csv(os.path.join(PROCESSED_DIR, 'image_download_actual.csv'))

In [ ]:
df_images.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,schools,country,schools_bin,is_train
0,4.270870351534024_6.253780978358008_4.31578611...,4.270870,6.253781,4.315786,6.268753,0.000000,0,ng,0,True
1,4.270870351534024_6.283724821161992_4.31578611...,4.270870,6.283725,4.315786,6.268753,0.000000,0,ng,0,True
2,4.330758037141992_6.298696742563984_4.31578611...,4.330758,6.298697,4.315786,6.268753,0.000000,0,ng,0,False
3,4.300814194338008_6.313668663965976_4.31578611...,4.300814,6.313669,4.315786,6.268753,0.000000,0,ng,0,True
4,4.283803377034025_6.3231437899519936_4.3287191...,4.283803,6.323144,4.328719,6.308172,0.295547,0,ng,0,True


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} as backend')
model = torch.load(CNN_DIR, map_location=device)

Using cpu as backend


In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=4, bias=True)
)

In [ ]:
# rip off the final layers
model.classifier = model.classifier[:4]

In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [ ]:
transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# custom dataset for fast image loading and processing
# does not follow the usual style of folder -> folder for each class -> image
# we just want one folder with images
class ForwardPassDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transformer):
        self.image_dir = image_dir
        self.image_list = os.listdir(self.image_dir)
        self.transformer = transformer

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image_name = self.image_list[index]

        # Load image
        X = self.filename_to_im_tensor(self.image_dir + '/' + image_name)
        
        # dataloaders need to return a label, but for the forward pass we don't really care
        return X, -1
    
    def filename_to_im_tensor(self, file):
        im = plt.imread(file)[:,:,:3]
        im = self.transformer(im)
        return im

model.eval()  
classes = [0, 1, 2, 3]
# shape of final array will be (num_validation_images, 4096)
# we also want to record the image each index represents
feats = np.zeros(((~df_images['is_train']).sum(), 4096))
image_order = []
i = 0
for c in classes:
    # use the validation images to do the forward pass
    dataset = ForwardPassDataset(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(c)), transformer)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
    image_order += dataset.image_list
    # forward pass for this class
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        feats[i:i+len(inputs),:] = outputs.cpu().detach().numpy()
        i += len(inputs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
feats

array([[-3.27435821e-01,  5.38410485e-01,  1.40602991e-01, ...,
        -1.06595004e+00, -8.74223828e-01,  5.20256102e-01],
       [-1.48112789e-01,  7.73098350e-01,  1.38936490e-01, ...,
        -2.78517038e-01, -5.94065607e-01, -5.21032751e-01],
       [ 5.68212047e-02, -9.39960331e-02,  3.60634327e-01, ...,
         8.33667040e-01, -1.25991607e+00, -1.12887633e+00],
       ...,
       [ 4.30705585e-02,  4.85358715e-01,  5.85950434e-01, ...,
        -7.95800924e-01, -8.27762127e-01,  4.65841964e-04],
       [ 6.16564274e-01, -1.12488818e+00,  2.91511744e-01, ...,
         3.33475858e-01, -1.41035628e+00, -1.28689241e+00],
       [ 2.82218361e+00, -1.49216354e-01, -1.77189231e-01, ...,
        -1.18138850e+00, -2.15659332e+00, -1.12570977e+00]])

In [ ]:
forward_pass_df = pd.DataFrame.from_dict({'image_name': image_order, 'feat_index': np.arange(len(image_order))})
forward_pass_df.head()

,image_name,feat_index
0,4.330758037141992_6.298696742563984_4.31578611...,0
1,4.373634905445978_6.338115711353986_4.32871914...,1
2,4.758727279761992_6.732184091724024_4.74375535...,2
3,4.698839594154023_6.77709985593_4.74375535836_...,3
4,4.75394354132_6.6988931604760165_4.75394354132...,4


In [ ]:
df_consumption = pd.merge(left=df_images, right=forward_pass_df, on='image_name')

In [ ]:
# have we maintained all validation images?
assert len(df_consumption) == (~df_images['is_train']).sum()

In [ ]:
df_consumption.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,schools,country,schools_bin,is_train,feat_index
0,4.330758037141992_6.298696742563984_4.31578611...,4.330758,6.298697,4.315786,6.268753,0.000000,0,ng,0,False,0
1,4.373634905445978_6.338115711353986_4.32871914...,4.373635,6.338116,4.328719,6.308172,0.295547,0,ng,0,False,1
2,4.368482925876016_7.139046639044024_4.39842676...,4.368483,7.139047,4.398427,7.183962,0.232205,32,ng,2,False,291
3,4.413398690081992_7.1540185604460165_4.3984267...,4.413399,7.154019,4.398427,7.183962,0.232205,32,ng,2,False,292
4,4.383454847278008_7.168990481848009_4.39842676...,4.383455,7.168990,4.398427,7.183962,0.232205,32,ng,2,False,293


## Aggregate Features
For each country, we aggregate the image features per cluster and save them to results/country/cnn

In [ ]:
country_abbrv = ['mw', 'eth', 'ng']
country_dir = ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']

for ca, cd in zip(country_abbrv, country_dir):
    df_c = df_consumption[df_consumption['country'] == ca]
    group = df_c.groupby(['cluster_lat', 'cluster_lon'])
    x = np.zeros((len(group), 4096))
    cluster_list = [] # the corresponding clusters (lat, lon) to the x aggregate feature array
    for i, g in enumerate(group):
        lat, lon = g[0]
        im_sub = df_consumption[(df_consumption['cluster_lat'] == lat) & (df_consumption['cluster_lon'] == lon)].reset_index(drop=True)
        agg_feats = np.zeros((len(im_sub), 4096))
        for j, d in im_sub.iterrows():
            agg_feats[j,:] = feats[d.feat_index]
        agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster

        x[i,:] = agg_feats
        cluster_list.append([lat, lon])
    # save to the correct directory
    save_dir = os.path.join(RESULTS_DIR, cd, 'cnn')
    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, 'cluster_feats.npy'), x)
    pickle.dump(cluster_list, open(os.path.join(save_dir, 'cluster_order.pkl'), 'wb')) 
    